In [1]:
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import pipeline
import numpy as np
import pandas as pd
import torch

In [2]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipe = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
)


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# model_name = "meta-llama/Meta-Llama-3-8B"
# model = AutoModelForCausalLM.from_pretrained(model_name,
#                                              torch_dtype=torch.bfloat16,
#                                              device_map="auto"
#                                             )
# tokenizer = AutoTokenizer.from_pretrained('ZWG817/Llama3_Chat_Materials')
# model.resize_token_embeddings(len(tokenizer))

# model.load_adapter('ZWG817/Llama3_Chat_Materials')

In [4]:
data = load_dataset("ZWG817/Materials_Gilbert_Damping")
data_train = data["train"]

In [5]:
messages = [
    {"role": "system", "content": "You are a helpful assistant, Read the following text. Does it give the Gilbert damping constant of a certain material? If so, list the corresponding material and its Gilbert damping canstant. Make your answer include the material's molecular formula, and make your answer as short as possible. and them, please output the original sentence as well"},
    {"role": "user", "content": data_train[4]['content'][:20000]},
]

terminators = [
    pipe.tokenizer.eos_token_id,
    pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipe(
    messages,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
assistant_response = outputs[-1]["generated_text"][-1]["content"]
print(assistant_response)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


The Gilbert damping constant is mentioned in the text as α = 0.0002, which represents the damping of the spin-waves mainly into the phonon system.

The material mentioned in the text is Yttrium Iron Garnet (YIG), with the molecular formula Y3Fe5O12.


In [12]:
df = pd.DataFrame()
df_tmp = pd.DataFrame()
df['Damping Constant'] = []
results = []

for i,j in enumerate(data_train):
    print(i)
    messages = [
        {"role": "system", "content": "You are a helpful assistant, Read the following text. Does it give the Gilbert damping constant of a certain material? If so, list the corresponding material and its Gilbert damping canstant. Make your answer include the material's molecular formula, and make your answer as short as possible. Please Answer me with format as follows: \nChemical Formula: \nDamping Constant: \nOriginal Sentences: \nIf the paper does not mention Gilbert Constant, Please just output: No Mention"},
        {"role": "user", "content": data_train[i]['content'][:20000]},
    ]
    
    terminators = [
        pipe.tokenizer.eos_token_id,
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    try:
        outputs = pipe(
            messages,
            max_new_tokens=2048,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
        )

        assistant_response = outputs[-1]["generated_text"][-1]["content"]
        print(assistant_response)
        results.append(assistant_response)
        
    except:
        pass

    if len(results) % 1000 == 0:
        df_tmp['Damping Constant'] = results
        df = pd.concat([df,df_tmp])
        results = []
        df_tmp = pd.DataFrame()
        df.to_csv('Gilbert Damping Constant.csv',index = False)

    df_tmp['Damping Constant'] = results
    df = pd.concat([df,df_tmp])
    df.to_csv('Gilbert Damping Constant.csv',index = False)
    
    # if i >= 20:
    #     break
    

    
    

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


0


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


No Mention
1


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

No Mention
18


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


No Mention
19


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


No Mention
20
Chemical Formula: Co1.5Fe1.5Ge

Damping Constant: 0.007 60.001

Original Sentences: The evolution of the linewidth with the resonance frequency indicates slight contributions of the inhomogeneous broadening, whose influence is gradually suppressed for frequencies up to 17 GHz, above which the subsequent linewidth evolution is consistent with a Gilbert damping aof 0.007 60.001.


In [11]:
df

,Damping Constant
0,The text does not explicitly provide the Gilbe...
1,The Gilbert damping constant is not mentioned ...
2,The Gilbert damping constant of Cu2OSeO3 is li...
3,The text does not explicitly give the Gilbert ...
4,The Gilbert damping constant is mentioned in t...
0,The Gilbert damping constant is not mentioned ...
1,The text does not explicitly provide the Gilbe...
2,The text does not provide the Gilbert damping ...
3,The text does not give the Gilbert damping con...
4,The text does not explicitly provide the Gilbe...
